# 快速开始

介绍如何使用 nwpc-data 库。

首先导入需要使用的一些库。

In [26]:
import numpy as np
import pandas as pd
import eccodes

## 获取业务系统产品文件路径

可以使用内置的配置文件获取 CMA-PI 或二级存储中业务系统产品文件路径。

下面的示例获取 GRAPES GFS GMF 系统 24 小时的原始分辨率 GRIB 2 文件路径。

In [27]:
from nwpc_data.data_finder import find_local_file

file_path = find_local_file(
    "grapes_gfs_gmf/grib2/orig",
    start_time="2020042200",
    forecast_time="24h"
)
file_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020042121/ORIG/gmf.gra.2020042200024.grb2')

使用 `number` 参数指定集合预报的成员编号。

In [28]:
file_path = find_local_file(
    "grapes_geps/grib2/orig",
    start_time="2020042200",
    forecast_time="24h",
    number=20
)
file_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GEPS/Prod-grib/2020042200/grib2/gef.gra.020.2020042200024.grb2')

nwpc-data 还内置了共享数据集的部分数据。

获取 FNL 资料。

In [29]:
file_path = find_local_file(
    "glob/fnl",
    start_time="2019122000",
    data_class="cm",
)
file_path

PosixPath('/sstorage1/COMMONDATA/glob/fnl/2019/fnl_20191220_00_00.grib2')

内置的配置文件请查看 `nwpc_data/data_finder/conf` 目录

## 检索 GRIB 2 消息

nwpc-data 使用 [eccodes-python](https://github.com/ecmwf/eccodes-python) 从 GRIB 2 文件中检索需要的要素场，并返回 GRIB 2 消息。

下面的示例从文件中检索单个要素场。

In [30]:
data_path = find_local_file(
    "grapes_gfs_gmf/grib2/orig",
    start_time="2020042200",
    forecast_time="24h"
)
data_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020042121/ORIG/gmf.gra.2020042200024.grb2')

In [31]:
from nwpc_data.grib.eccodes import load_message_from_file

t850 = load_message_from_file(
    data_path,
    parameter="t",
    level_type="pl",
    level=850,
)
t850

94357887418928

可以使用 eccodes-python 的 API 接口获取 GRIB 2 消息的属性和数据。

In [32]:
print("shortName:   ", eccodes.codes_get(t850, "shortName"))
print("typeOfLevel: ", eccodes.codes_get(t850, "typeOfLevel"))
print("level:       ", eccodes.codes_get(t850, "level"))

shortName:    t
typeOfLevel:  isobaricInhPa
level:        850


In [33]:
values = eccodes.codes_get_double_array(t850, "values")
values = values.reshape([720, 1440])
values

array([[258.75289063, 258.76289063, 258.76289063, ..., 258.75289063,
        258.76289063, 258.75289063],
       [258.47289062, 258.48289062, 258.46289062, ..., 258.46289062,
        258.47289062, 258.44289063],
       [258.27289063, 258.26289063, 258.27289063, ..., 258.25289063,
        258.26289063, 258.26289063],
       ...,
       [227.68289062, 227.68289062, 227.72289062, ..., 227.62289062,
        227.75289062, 227.63289063],
       [227.90289062, 227.90289062, 227.90289062, ..., 227.90289062,
        227.88289063, 227.91289063],
       [228.12289062, 228.18289062, 228.19289063, ..., 228.09289062,
        228.23289063, 228.20289063]])

**注意**：需要手动调用 `eccodes.codes_release` 释放消息。

In [34]:
eccodes.codes_release(t850)

## 加载 GRIB 2 要素场

nwpc-data 还提供对上述检索得到 GRIB 2 消息的封装，返回 `xarray.DataArray` 对象，类似 [cfgrib](https://github.com/ecmwf/cfgrib) 库。

In [35]:
from nwpc_data.grib.eccodes import load_field_from_file

t850 = load_field_from_file(
    data_path,
    parameter="t",
    level_type="pl",
    level=850,
)
t850

<xarray.DataArray 't' (latitude: 720, longitude: 1440)>
array([[258.75289063, 258.76289063, 258.76289063, ..., 258.75289063,
        258.76289063, 258.75289063],
       [258.47289062, 258.48289062, 258.46289062, ..., 258.46289062,
        258.47289062, 258.44289063],
       [258.27289063, 258.26289063, 258.27289063, ..., 258.25289063,
        258.26289063, 258.26289063],
       ...,
       [227.68289062, 227.68289062, 227.72289062, ..., 227.62289062,
        227.75289062, 227.63289063],
       [227.90289062, 227.90289062, 227.90289062, ..., 227.90289062,
        227.88289063, 227.91289063],
       [228.12289062, 228.18289062, 228.19289063, ..., 228.09289062,
        228.23289063, 228.20289063]])
Coordinates:
    time        datetime64[ns] 2020-04-22
    step        timedelta64[ns] 1 days
    valid_time  datetime64[ns] 2020-04-23
    pl          float64 850.0
  * latitude    (latitude) float64 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Attributes:
    GRIB_edition:                    2
    GRIB_centre:                     babj
    GRIB_subCentre:                  0
    GRIB_tablesVersion:              4
    GRIB_localTablesVersion:         1
    GRIB_dataType:                   fc
    GRIB_dataDate:                   20200422
    GRIB_dataTime:                   0
    GRIB_validityDate:               20200423
    GRIB_validityTime:               0
    GRIB_step:                       24
    GRIB_stepType:                   instant
    GRIB_stepUnits:                  1
    GRIB_stepRange:                  24
    GRIB_endStep:                    24
    GRIB_name:                       Temperature
    GRIB_shortName:                  t
    GRIB_cfName:                     air_temperature
    GRIB_discipline:                 0
    GRIB_parameterCategory:          0
    GRIB_parameterNumber:            0
    GRIB_gridType:                   regular_ll
    GRIB_gridDefinitionDescription:  Latitude/longitude 
    GRIB_typeOfFirstFixedSurface:    pl
    GRIB_typeOfLevel:                isobaricInhPa
    GRIB_level:                      850
    GRIB_numberOfPoints:             1036800
    GRIB_missingValue:               9999
    GRIB_units:                      K
    long_name:                       Temperature
    units:                           K

## 下一步

请浏览下一章节 [介绍和安装](./01_introduction.ipynb)。